# Clustering using nearest neighbor
    -> Word count model
    -> tf-idf model
### Steps
    -> count and tfidf column
    -> can compute cosine similarity between anyone
    -> matrix to nearest neighbor
    -> fit
    -> predict => distances, indices

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data=pd.read_csv('people_wiki.csv')
data.head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


In [3]:
obama=data[data['name']=='Barack Obama']
#obama['text'].values

In [4]:
clooney=data[data['name']=='George Clooney']
clooney['text']

38514    george timothy clooney born may 6 1961 is an a...
Name: text, dtype: object

In [5]:
beckham=data[data['name']=='David Beckham']
a=data[data['name']=='Elton John']
b=data[data['name']=='Cliff Richard']
c=data[data['name']=='Roger Daltrey']
d=data[data['name']=='George Bush']
e=data[data['name']=='Rod Stewart']
f=data[data['name']=='Tommy Haas']
g=data[data['name']=='Elvis Presley']
h=data[data['name']=='Stephen Dow Beckham']
i=data[data['name']=='Louis Molloy']
j=data[data['name']=='Adrienne Corri']
k=data[data['name']=='Mary Fitzgerald(artist)']
l=data[data['name']=='Mel B']
m=data[data['name']=='Caroline Rush']
n=data[data['name']=='Carrie Reichardt']
o=data[data['name']=='Victoria Beckham']
p=data[data['name']=='Paul McCartney']
q=data[data['name']=='Bill Clinton']

In [6]:
data=data[:300]
data.shape

(300, 3)

In [7]:
type(obama)

pandas.core.frame.DataFrame

In [8]:
data=data.append(obama,ignore_index=True)
data=data.append(clooney,ignore_index=True)
data=data.append(beckham,ignore_index=True)
data=data.append(a,ignore_index=True)
data=data.append(b,ignore_index=True)
data=data.append(c,ignore_index=True)
data=data.append(d,ignore_index=True)
data=data.append(e,ignore_index=True)
data=data.append(f,ignore_index=True)
data=data.append(g,ignore_index=True)
data=data.append(h,ignore_index=True)
data=data.append(i,ignore_index=True)
data=data.append(j,ignore_index=True)
data=data.append(k,ignore_index=True)
data=data.append(l,ignore_index=True)
data=data.append(m,ignore_index=True)
data=data.append(n,ignore_index=True)
data=data.append(o,ignore_index=True)
data=data.append(p,ignore_index=True)
data=data.append(q,ignore_index=True)

In [9]:
data.tail(18)

,URI,name,text
299,<http://dbpedia.org/resource/Jeff_Juden>,Jeff Juden,jeffrey daniel juden born january 19 1971 in s...
300,<http://dbpedia.org/resource/Barack_Obama>,Barack Obama,barack hussein obama ii brk husen bm born augu...
301,<http://dbpedia.org/resource/George_Clooney>,George Clooney,george timothy clooney born may 6 1961 is an a...
302,<http://dbpedia.org/resource/David_Beckham>,David Beckham,david robert joseph beckham obe bkm born 2 may...
303,<http://dbpedia.org/resource/Elton_John>,Elton John,sir elton hercules john cbe born reginald kenn...
304,<http://dbpedia.org/resource/Cliff_Richard>,Cliff Richard,sir cliff richard obe born harry rodger webb 1...
305,<http://dbpedia.org/resource/Roger_Daltrey>,Roger Daltrey,roger harry daltrey cbe born 1 march 1944 is a...
306,<http://dbpedia.org/resource/Rod_Stewart>,Rod Stewart,roderick david rod stewart cbe born 10 january...
307,<http://dbpedia.org/resource/Tommy_Haas>,Tommy Haas,thomas mario tommy haas born 3 april 1978 is a...
308,<http://dbpedia.org/resource/Stephen_Dow_Beckham>,Stephen Dow Beckham,stephen dow beckham is a noted american histor...


In [10]:
data.shape

(317, 3)

## obama count top words

In [11]:
obama=data[data['name']=='Barack Obama']
obama['text'].values

array(['barack hussein obama ii brk husen bm born august 4 1961 is the 44th and current president of the united states and the first african american to hold the office born in honolulu hawaii obama is a graduate of columbia university and harvard law school where he served as president of the harvard law review he was a community organizer in chicago before earning his law degree he worked as a civil rights attorney and taught constitutional law at the university of chicago law school from 1992 to 2004 he served three terms representing the 13th district in the illinois senate from 1997 to 2004 running unsuccessfully for the united states house of representatives in 2000in 2004 obama received national attention during his campaign to represent illinois in the united states senate with his victory in the march democratic party primary his keynote address at the democratic national convention in july and his election to the senate in november he began his presidential campaign in 2007 a

In [12]:
cv=CountVectorizer()

In [13]:
cv_fit=cv.fit_transform(obama['text'])
cv.vocabulary_

{'barack': 36,
 'hussein': 112,
 'obama': 160,
 'ii': 113,
 'brk': 43,
 'husen': 111,
 'bm': 40,
 'born': 41,
 'august': 35,
 '1961': 1,
 'is': 124,
 'the': 242,
 '44th': 16,
 'and': 28,
 'current': 65,
 'president': 179,
 'of': 162,
 'united': 252,
 'states': 228,
 'first': 90,
 'african': 23,
 'american': 26,
 'to': 245,
 'hold': 107,
 'office': 163,
 'in': 115,
 'honolulu': 108,
 'hawaii': 103,
 'graduate': 98,
 'columbia': 54,
 'university': 253,
 'harvard': 101,
 'law': 133,
 'school': 217,
 'where': 261,
 'he': 104,
 'served': 221,
 'as': 30,
 'review': 210,
 'was': 260,
 'community': 56,
 'organizer': 170,
 'chicago': 50,
 'before': 37,
 'earning': 81,
 'his': 106,
 'degree': 72,
 'worked': 268,
 'civil': 51,
 'rights': 211,
 'attorney': 34,
 'taught': 235,
 'constitutional': 57,
 'at': 32,
 'from': 94,
 '1992': 2,
 '2004': 7,
 'three': 244,
 'terms': 240,
 'representing': 205,
 '13th': 0,
 'district': 75,
 'illinois': 114,
 'senate': 220,
 '1997': 4,
 'running': 214,
 'unsucces

In [14]:
words=cv.get_feature_names()
counts=cv_fit.toarray()
print(words)

['13th', '1961', '1992', '1996', '1997', '20', '2000in', '2004', '2007', '2008', '2009', '2010', '2011', '2012', '2012obama', '2013', '44th', '63', 'act', 'address', 'administration', 'affordable', 'afghanistan', 'african', 'after', 'against', 'american', 'americans', 'and', 'arms', 'as', 'ask', 'at', 'attention', 'attorney', 'august', 'barack', 'before', 'began', 'bin', 'bm', 'born', 'briefs', 'brk', 'budget', 'by', 'californias', 'called', 'campaign', 'care', 'chicago', 'civil', 'clinton', 'close', 'columbia', 'combat', 'community', 'constitutional', 'consumer', 'continued', 'control', 'convention', 'court', 'creation', 'cuba', 'current', 'death', 'debate', 'debt', 'defeated', 'defeating', 'defense', 'degree', 'delegates', 'democratic', 'district', 'doddfrank', 'domestic', 'dont', 'down', 'during', 'earning', 'economic', 'election', 'elementary', 'ended', 'ending', 'equality', 'federal', 'filed', 'first', 'for', 'foreign', 'form', 'from', 'full', 'gains', 'general', 'graduate', 'grea

In [15]:
print(counts)

[[ 1  1  1  1  1  2  1  3  1  1  3  2  3  1  1  1  1  1  8  1  1  1  2  1
   4  1  3  1 21  1  6  1  2  1  1  1  1  1  1  1  1  2  1  1  1  1  1  1
   3  1  2  1  1  1  1  1  1  1  1  1  4  1  1  1  1  1  1  1  1  1  1  1
   1  1  4  1  1  2  2  1  2  1  1  3  1  1  1  1  1  1  3  4  2  1  3  1
   1  1  1  1  1  2  4  1  7  1 11  1  1  1  2  1  1  1  2 30  1  1  1  1
   1  1  3  4  2  1  3  1  1  1  1  1  1  6  1  1  1  1  1  1  1  1  1  1
   1  1  4  1  1  1  2  1  1  1  1  1  2  1  1  2  9  1 18  2  1  2  1  1
   1  3  1  1  1  1  3  1  1  1  2  4  2  1  2  1  1  1  1  2  1  1  1  1
   1  1  1  1  1  1  1  1  1  2  1  1  2  1  2  1  3  1  1  1  1  1  1  1
   1  3  1  2  3  2  1  3  1  1  1  1  3  1  1  1  1  1  1  1  1  1  1  3
   1  1 40  1  1 14  1  1  1  1  1  1  3  2  1  1  6  1  1  1  5  1  1  1
   1  3  1  1  1  1]]


In [16]:
print(counts.shape)
print(type(counts))
counts=counts.reshape(270)
counts.shape

(1, 270)
<class 'numpy.ndarray'>


(270,)

In [17]:
z=zip(words,counts)
z=dict(z)
z

{'13th': 1,
 '1961': 1,
 '1992': 1,
 '1996': 1,
 '1997': 1,
 '20': 2,
 '2000in': 1,
 '2004': 3,
 '2007': 1,
 '2008': 1,
 '2009': 3,
 '2010': 2,
 '2011': 3,
 '2012': 1,
 '2012obama': 1,
 '2013': 1,
 '44th': 1,
 '63': 1,
 'act': 8,
 'address': 1,
 'administration': 1,
 'affordable': 1,
 'afghanistan': 2,
 'african': 1,
 'after': 4,
 'against': 1,
 'american': 3,
 'americans': 1,
 'and': 21,
 'arms': 1,
 'as': 6,
 'ask': 1,
 'at': 2,
 'attention': 1,
 'attorney': 1,
 'august': 1,
 'barack': 1,
 'before': 1,
 'began': 1,
 'bin': 1,
 'bm': 1,
 'born': 2,
 'briefs': 1,
 'brk': 1,
 'budget': 1,
 'by': 1,
 'californias': 1,
 'called': 1,
 'campaign': 3,
 'care': 1,
 'chicago': 2,
 'civil': 1,
 'clinton': 1,
 'close': 1,
 'columbia': 1,
 'combat': 1,
 'community': 1,
 'constitutional': 1,
 'consumer': 1,
 'continued': 1,
 'control': 4,
 'convention': 1,
 'court': 1,
 'creation': 1,
 'cuba': 1,
 'current': 1,
 'death': 1,
 'debate': 1,
 'debt': 1,
 'defeated': 1,
 'defeating': 1,
 'defense': 1,


In [18]:
z_sort=sorted(z.items(),key=lambda kv : (kv[1],kv[0]), reverse=True )
z_sort

[('the', 40),
 ('in', 30),
 ('and', 21),
 ('of', 18),
 ('to', 14),
 ('his', 11),
 ('obama', 9),
 ('act', 8),
 ('he', 7),
 ('us', 6),
 ('law', 6),
 ('as', 6),
 ('was', 5),
 ('president', 4),
 ('military', 4),
 ('iraq', 4),
 ('has', 4),
 ('for', 4),
 ('democratic', 4),
 ('control', 4),
 ('after', 4),
 ('with', 3),
 ('united', 3),
 ('term', 3),
 ('states', 3),
 ('signed', 3),
 ('senate', 3),
 ('school', 3),
 ('response', 3),
 ('party', 3),
 ('ordered', 3),
 ('january', 3),
 ('involvement', 3),
 ('from', 3),
 ('first', 3),
 ('election', 3),
 ('campaign', 3),
 ('american', 3),
 ('2011', 3),
 ('2009', 3),
 ('2004', 3),
 ('university', 2),
 ('served', 2),
 ('second', 2),
 ('republican', 2),
 ('representatives', 2),
 ('relief', 2),
 ('protection', 2),
 ('primary', 2),
 ('presidential', 2),
 ('policy', 2),
 ('on', 2),
 ('office', 2),
 ('november', 2),
 ('nominee', 2),
 ('national', 2),
 ('is', 2),
 ('illinois', 2),
 ('house', 2),
 ('harvard', 2),
 ('foreign', 2),
 ('during', 2),
 ('dont', 2),
 

## tfidf column

In [19]:
tf=TfidfVectorizer()

In [20]:
tf_fit=tf.fit_transform(data['text'])
words=tf.get_feature_names()
counts=tf_fit.toarray()
print(counts)

[[0.         0.         0.03670929 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.03452948 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [21]:
print(counts.shape)

(317, 15959)


In [22]:
d=[]
for i in range(len(counts)):
    mapped=zip(words,counts[i])
    mapped=dict(mapped)
    d.append(mapped)
len(d)

317

In [23]:
data['tfidf']=d
data.head()

,URI,name,text,tfidf
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...,"{'05': 0.0, '0fer': 0.0, '10': 0.0367092932515..."
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,"{'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.0..."
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...,"{'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.0..."
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,"{'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.0..."
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...,"{'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.0..."


## count column

In [24]:
cv=CountVectorizer()
cv_fit=cv.fit_transform(data['text'])
words1=cv.get_feature_names()
counts1=cv_fit.toarray()

In [25]:
d=[]
for i in range(len(counts1)):
    mapped=zip(words1,counts1[i])
    mapped=dict(mapped)
    d.append(mapped)
len(d)

317

In [26]:
data['count']=d
data.head()

,URI,name,text,tfidf,count
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...,"{'05': 0.0, '0fer': 0.0, '10': 0.0367092932515...","{'05': 0, '0fer': 0, '10': 1, '100': 0, '1000'..."
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,"{'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.0...","{'05': 0, '0fer': 0, '10': 0, '100': 0, '1000'..."
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...,"{'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.0...","{'05': 0, '0fer': 0, '10': 0, '100': 0, '1000'..."
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,"{'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.0...","{'05': 0, '0fer': 0, '10': 0, '100': 0, '1000'..."
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...,"{'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.0...","{'05': 0, '0fer': 0, '10': 0, '100': 0, '1000'..."


## obama tfidf top words

In [27]:
obama['tfidf']=data[data['name']=='Barack Obama']['tfidf']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
obama

,URI,name,text,tfidf
300,<http://dbpedia.org/resource/Barack_Obama>,Barack Obama,barack hussein obama ii brk husen bm born augu...,"{'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.0..."


In [29]:
tfidf=obama['tfidf']
type(tfidf)

pandas.core.series.Series

In [30]:
tfidf=tfidf.as_matrix()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [31]:
tfidf

array([{'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.0, '1000': 0.0, '100000': 0.0, '1000000': 0.0, '1000plus': 0.0, '100200and': 0.0, '100dbs': 0.0, '100m': 0.0, '100s': 0.0, '100th': 0.0, '1011': 0.0, '102': 0.0, '1021': 0.0, '1028': 0.0, '106': 0.0, '107': 0.0, '109': 0.0, '10m': 0.0, '10part': 0.0, '10th': 0.0, '11': 0.0, '110': 0.0, '1107': 0.0, '1111': 0.0, '1115': 0.0, '113': 0.0, '1135': 0.0, '115': 0.0, '1151': 0.0, '116': 0.0, '117': 0.0, '1184': 0.0, '11am': 0.0, '11th': 0.0, '12': 0.0, '120': 0.0, '120000': 0.0, '121': 0.0, '1211': 0.0, '1214153': 0.0, '122': 0.0, '123': 0.0, '1233': 0.0, '125th': 0.0, '129': 0.0, '129000': 0.0, '1298': 0.0, '12month': 0.0, '12page': 0.0, '12th': 0.0, '13': 0.0, '130': 0.0, '131': 0.0, '1313': 0.0, '135000': 0.0, '136': 0.0, '138': 0.0, '1391': 0.0, '13px': 0.0, '13th': 0.03831529897953652, '13yearold': 0.0, '14': 0.0, '140': 0.0, '1406': 0.0, '147': 0.0, '14th': 0.0, '14the': 0.0, '14yearolds': 0.0, '15': 0.0, '150': 0.0, '15000': 0.0, '150

In [32]:
tfidf=tfidf[0]

In [33]:
tfidf

{'05': 0.0,
 '0fer': 0.0,
 '10': 0.0,
 '100': 0.0,
 '1000': 0.0,
 '100000': 0.0,
 '1000000': 0.0,
 '1000plus': 0.0,
 '100200and': 0.0,
 '100dbs': 0.0,
 '100m': 0.0,
 '100s': 0.0,
 '100th': 0.0,
 '1011': 0.0,
 '102': 0.0,
 '1021': 0.0,
 '1028': 0.0,
 '106': 0.0,
 '107': 0.0,
 '109': 0.0,
 '10m': 0.0,
 '10part': 0.0,
 '10th': 0.0,
 '11': 0.0,
 '110': 0.0,
 '1107': 0.0,
 '1111': 0.0,
 '1115': 0.0,
 '113': 0.0,
 '1135': 0.0,
 '115': 0.0,
 '1151': 0.0,
 '116': 0.0,
 '117': 0.0,
 '1184': 0.0,
 '11am': 0.0,
 '11th': 0.0,
 '12': 0.0,
 '120': 0.0,
 '120000': 0.0,
 '121': 0.0,
 '1211': 0.0,
 '1214153': 0.0,
 '122': 0.0,
 '123': 0.0,
 '1233': 0.0,
 '125th': 0.0,
 '129': 0.0,
 '129000': 0.0,
 '1298': 0.0,
 '12month': 0.0,
 '12page': 0.0,
 '12th': 0.0,
 '13': 0.0,
 '130': 0.0,
 '131': 0.0,
 '1313': 0.0,
 '135000': 0.0,
 '136': 0.0,
 '138': 0.0,
 '1391': 0.0,
 '13px': 0.0,
 '13th': 0.03831529897953652,
 '13yearold': 0.0,
 '14': 0.0,
 '140': 0.0,
 '1406': 0.0,
 '147': 0.0,
 '14th': 0.0,
 '14the': 0.0

In [34]:
z_sort=sorted(tfidf.items(),key=lambda kv: (kv[1],kv[0]), reverse=True )

In [35]:
z_sort

[('obama', 0.379024599948989),
 ('the', 0.2974436114802383),
 ('act', 0.2596180708833073),
 ('in', 0.22378533189529953),
 ('and', 0.15615789602712513),
 ('iraq', 0.14783815768779676),
 ('military', 0.13928123822897903),
 ('law', 0.13711430357432638),
 ('democratic', 0.13577785707930498),
 ('of', 0.13427119913717975),
 ('us', 0.12991907653866858),
 ('control', 0.12980903544165365),
 ('involvement', 0.1199238437222164),
 ('response', 0.11087861826584758),
 ('ordered', 0.11087861826584758),
 ('to', 0.11086772295982786),
 ('his', 0.10115003443314549),
 ('senate', 0.09197686809636892),
 ('president', 0.08995830345269064),
 ('campaign', 0.08899801362125215),
 ('term', 0.08764558313692435),
 ('nominee', 0.08422768887755312),
 ('election', 0.08402006683764532),
 ('signed', 0.0773252523026278),
 ('representatives', 0.07663059795907304),
 ('protection', 0.07663059795907304),
 ('dont', 0.07663059795907304),
 ('afghanistan', 0.07391907884389838),
 ('party', 0.0728486360743893),
 ('relief', 0.06964

## elton john count and tfidf top words

In [36]:
john=data[data['name']=='Elton John']
john

,URI,name,text,tfidf,count
303,<http://dbpedia.org/resource/Elton_John>,Elton John,sir elton hercules john cbe born reginald kenn...,"{'05': 0.0, '0fer': 0.0, '10': 0.0304392031560...","{'05': 0, '0fer': 0, '10': 1, '100': 3, '1000'..."


In [37]:
j_count=john['count']
j_count

303    {'05': 0, '0fer': 0, '10': 1, '100': 3, '1000'...
Name: count, dtype: object

In [38]:
j_tf=john['tfidf']
j_tf

303    {'05': 0.0, '0fer': 0.0, '10': 0.0304392031560...
Name: tfidf, dtype: object

In [39]:
j_count=j_count.as_matrix()
j_count=j_count[0]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [40]:
j_tf=j_tf.as_matrix()
j_tf=j_tf[0]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [41]:
z1=sorted(j_count.items(),key=lambda kv: (kv[1],kv[0]), reverse=True )
z1

[('the', 27),
 ('in', 18),
 ('and', 15),
 ('of', 13),
 ('has', 9),
 ('john', 7),
 ('he', 7),
 ('on', 6),
 ('since', 5),
 ('for', 5),
 ('award', 5),
 ('top', 4),
 ('to', 4),
 ('is', 4),
 ('his', 4),
 ('billboard', 4),
 ('at', 4),
 ('than', 3),
 ('one', 3),
 ('no', 3),
 ('music', 3),
 ('more', 3),
 ('million', 3),
 ('into', 3),
 ('him', 3),
 ('elton', 3),
 ('british', 3),
 ('been', 3),
 ('awards', 3),
 ('an', 3),
 ('academy', 3),
 ('100', 3),
 ('worldwide', 2),
 ('with', 2),
 ('was', 2),
 ('songwriters', 2),
 ('songwriter', 2),
 ('sold', 2),
 ('single', 2),
 ('services', 2),
 ('roll', 2),
 ('rock', 2),
 ('received', 2),
 ('ranked', 2),
 ('party', 2),
 ('palace', 2),
 ('over', 2),
 ('number', 2),
 ('marriage', 2),
 ('its', 2),
 ('hot', 2),
 ('hall', 2),
 ('gay', 2),
 ('furnish', 2),
 ('foundation', 2),
 ('fame', 2),
 ('december', 2),
 ('consecutive', 2),
 ('bestselling', 2),
 ('as', 2),
 ('artists', 2),
 ('albums', 2),
 ('aids', 2),
 ('40', 2),
 ('21', 2),
 ('2004', 2),
 ('1997', 2),
 ('y

In [42]:
z2=sorted(j_tf.items(),key=lambda kv: (kv[1],kv[0]), reverse=True )
z2

[('the', 0.27174429328375915),
 ('john', 0.2066652579797534),
 ('billboard', 0.1837726865493523),
 ('in', 0.18173345435279797),
 ('elton', 0.1710008060205663),
 ('and', 0.15096905182431067),
 ('award', 0.1325969735946084),
 ('has', 0.13168580672358923),
 ('of', 0.13125193925479853),
 ('since', 0.12235125985287831),
 ('songwriters', 0.12216222840290171),
 ('furnish', 0.12216222840290171),
 ('top', 0.11555095076512659),
 ('million', 0.10949957937323158),
 ('100', 0.10949957937323158),
 ('palace', 0.10820972538639115),
 ('gay', 0.10004803433053368),
 ('aids', 0.10004803433053368),
 ('academy', 0.09952832450528921),
 ('bestselling', 0.09694510340098778),
 ('british', 0.09682259197942684),
 ('awards', 0.09355521152589771),
 ('songwriter', 0.09188634327467615),
 ('hot', 0.09188634327467615),
 ('marriage', 0.08976551965486516),
 ('no', 0.08857082484846575),
 ('roll', 0.08784700132496337),
 ('ranked', 0.08609553131402312),
 ('than', 0.08545529958693604),
 ('he', 0.0846391710800144),
 ('worldwi

## compute cosine distance => obama clinton beckham

In [299]:
doc_matrix=cv_fit.todense()
df=pd.DataFrame(doc_matrix,columns=words1)
df.head()

,05,0fer,10,100,1000,100000,1000000,1000plus,100200and,100dbs,...,zubaydah,zubaydahs,zubin,zulu,zuni,zunnit,zur,zurich,zvonimir,zwigoff
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [300]:
from sklearn.metrics.pairwise import cosine_similarity

In [301]:
obama

,URI,name,text,tfidf
300,<http://dbpedia.org/resource/Barack_Obama>,Barack Obama,barack hussein obama ii brk husen bm born augu...,"{'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.0..."


In [302]:
a=list(obama['tfidf'].values[0].values())
a=pd.Series(a)
#a

In [303]:
df1=pd.DataFrame(columns=words1)
df1

,05,0fer,10,100,1000,100000,1000000,1000plus,100200and,100dbs,...,zubaydah,zubaydahs,zubin,zulu,zuni,zunnit,zur,zurich,zvonimir,zwigoff


In [308]:
df1=df1.append(a,ignore_index=True)

In [309]:
clinton=data[data['name']=='Bill Clinton']
clinton

,URI,name,text,tfidf,count
316,<http://dbpedia.org/resource/Bill_Clinton>,Bill Clinton,william jefferson bill clinton born william je...,"{'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.0...","{'05': 0, '0fer': 0, '10': 0, '100': 0, '1000'..."


In [310]:
b=list(clinton['tfidf'].values[0].values())
b=pd.Series(b)
df1=df1.append(b,ignore_index=True)

In [311]:
df1

,05,0fer,10,100,1000,100000,1000000,1000plus,100200and,100dbs,...,15949,15950,15951,15952,15953,15954,15955,15956,15957,15958
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [312]:
beckham=data[data['name']=='David Beckham']
beckham

,URI,name,text,tfidf,count
302,<http://dbpedia.org/resource/David_Beckham>,David Beckham,david robert joseph beckham obe bkm born 2 may...,"{'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.0...","{'05': 0, '0fer': 0, '10': 0, '100': 2, '1000'..."


In [313]:
b=list(beckham['tfidf'].values[0].values())
b=pd.Series(b)
df1=df1.append(b,ignore_index=True)

In [314]:
df1

,05,0fer,10,100,1000,100000,1000000,1000plus,100200and,100dbs,...,15949,15950,15951,15952,15953,15954,15955,15956,15957,15958
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [316]:
df1.fillna(0,inplace=True)

In [317]:
df1

,05,0fer,10,100,1000,100000,1000000,1000plus,100200and,100dbs,...,15949,15950,15951,15952,15953,15954,15955,15956,15957,15958
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [319]:
a=cosine_similarity(df1,df1)
a

array([[1.        , 0.36365984, 0.20044101],
       [0.36365984, 1.        , 0.2011385 ],
       [0.20044101, 0.2011385 , 1.        ]])

In [323]:
#obama #clinton #beckham
obama_clinton=a[0][1]
obama_beckham=a[0][2]
print(obama_clinton)
print(obama_beckham)
#obama is more realted to clinton than beckham

0.3636598416939041
0.20044101149969878


## compute cosine distance => elton john

In [325]:
elton=data[data['name']=='Elton John']
elton

,URI,name,text,tfidf,count
303,<http://dbpedia.org/resource/Elton_John>,Elton John,sir elton hercules john cbe born reginald kenn...,"{'05': 0.0, '0fer': 0.0, '10': 0.0304392031560...","{'05': 0, '0fer': 0, '10': 1, '100': 3, '1000'..."


In [329]:
victoria=data[data['name']=='Victoria Beckham']
victoria

,URI,name,text,tfidf,count
314,<http://dbpedia.org/resource/Victoria_Beckham>,Victoria Beckham,victoria caroline beckham ne adams born 17 apr...,"{'05': 0.0, '0fer': 0.0, '10': 0.0345294788899...","{'05': 0, '0fer': 0, '10': 1, '100': 0, '1000'..."


In [331]:
paul=data[data['name']=='Paul McCartney']
paul

,URI,name,text,tfidf,count
315,<http://dbpedia.org/resource/Paul_McCartney>,Paul McCartney,sir james paul mccartney mbe born 18 june 1942...,"{'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.1...","{'05': 0, '0fer': 0, '10': 0, '100': 3, '1000'..."


In [335]:
df2=pd.DataFrame(columns=words1)

In [336]:
b=list(elton['tfidf'].values[0].values())
b=pd.Series(b)
df2=df2.append(b,ignore_index=True)

In [338]:
b=list(victoria['tfidf'].values[0].values())
b=pd.Series(b)
df2=df2.append(b,ignore_index=True)

In [339]:
b=list(paul['tfidf'].values[0].values())
b=pd.Series(b)
df2=df2.append(b,ignore_index=True)

In [342]:
df2.fillna(0,inplace=True)

In [343]:
df2

,05,0fer,10,100,1000,100000,1000000,1000plus,100200and,100dbs,...,15949,15950,15951,15952,15953,15954,15955,15956,15957,15958
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [345]:
a=cosine_similarity(df2,df2)
a

array([[1.        , 0.19740861, 0.35810773],
       [0.19740861, 1.        , 0.17232955],
       [0.35810773, 0.17232955, 1.        ]])

In [346]:
# elton victoria paul
elton_victoria=a[0][1]
elton_paul=a[0][2]
print(elton_victoria)
print(elton_paul)
# elton is more related to paul than victoria

0.19740861460555725
0.3581077315029874


## nearest neighbor model using counts

In [351]:
from sklearn.neighbors import NearestNeighbors

In [356]:
nbrs=NearestNeighbors()

In [357]:
counts1

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [358]:
nbrs_fit=nbrs.fit(counts1)

In [363]:
distances,indices=nbrs_fit.kneighbors(counts1)
print(distances)
print(indices)

[[ 0.         22.42766149 24.75883681 25.29822128 25.51470164]
 [ 0.         22.3159136  22.38302929 23.57965225 23.93741841]
 [ 0.         22.36067977 22.42766149 22.60530911 23.15167381]
 ...
 [ 0.         24.41311123 25.05992817 25.0998008  25.23885893]
 [ 0.         22.8035085  24.22808288 24.39262184 25.55386468]
 [ 0.         34.14674216 35.17101079 36.34556369 36.61966685]]
[[  0  99 174 137 264]
 [  1 127  67 138  79]
 [  2 127  77 240  63]
 ...
 [314 230  91 233 309]
 [315 305  16  46 121]
 [316 183  36 218  41]]


In [370]:
a=nbrs_fit.kneighbors_graph(counts1).toarray()
a

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [371]:
a.shape

(317, 317)

In [372]:
a[1]  # 5 neighbors are 1

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [378]:
elton

,URI,name,text,tfidf,count
303,<http://dbpedia.org/resource/Elton_John>,Elton John,sir elton hercules john cbe born reginald kenn...,"{'05': 0.0, '0fer': 0.0, '10': 0.0304392031560...","{'05': 0, '0fer': 0, '10': 1, '100': 3, '1000'..."


In [381]:
nbrs_fit.kneighbors(counts1[[303]])

(array([[ 0.        , 27.74887385, 27.92848009, 30.95157508, 31.22498999]]),
 array([[303, 305, 306, 315,  27]], dtype=int64))

In [382]:
counts1[305]

array([0., 0., 0., ..., 0., 0., 0.])

In [384]:
data.loc[305]   # Elton john most related to Roger Daltrey

URI            <http://dbpedia.org/resource/Roger_Daltrey>
name                                         Roger Daltrey
text     roger harry daltrey cbe born 1 march 1944 is a...
tfidf    {'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.0...
count    {'05': 0, '0fer': 0, '10': 0, '100': 2, '1000'...
Name: 305, dtype: object

In [387]:
victoria

,URI,name,text,tfidf,count
314,<http://dbpedia.org/resource/Victoria_Beckham>,Victoria Beckham,victoria caroline beckham ne adams born 17 apr...,"{'05': 0.0, '0fer': 0.0, '10': 0.0345294788899...","{'05': 0, '0fer': 0, '10': 1, '100': 0, '1000'..."


In [388]:
nbrs_fit.kneighbors(counts1[[314]])

(array([[ 0.        , 24.41311123, 25.05992817, 25.0998008 , 25.23885893]]),
 array([[314, 230,  91, 233, 309]], dtype=int64))

In [389]:
data.loc[230]   # victoria most related to tony lundon

URI              <http://dbpedia.org/resource/Tony_Lundon>
name                                           Tony Lundon
text     anthony william james tony lundon born 13 apri...
tfidf    {'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.0...
count    {'05': 0, '0fer': 0, '10': 0, '100': 0, '1000'...
Name: 230, dtype: object

In [420]:
data.loc[309]

URI             <http://dbpedia.org/resource/Louis_Molloy>
name                                          Louis Molloy
text     louis molloy is a manchesterbased tattoo artis...
tfidf    {'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.0...
count    {'05': 0, '0fer': 0, '10': 0, '100': 0, '1000'...
Name: 309, dtype: object

## nearest neighbor model using tfidf

In [390]:
nbrs=NearestNeighbors()
nbrs_fit=nbrs.fit(counts)

In [391]:
distances,indices=nbrs_fit.kneighbors(counts1)
print(distances)
print(indices)

[[37.17895227 37.48173939 37.50110285 37.51532272 37.52121642]
 [21.83373877 22.33017135 22.36939625 22.37096887 22.37302616]
 [25.07053173 25.43399609 25.48931507 25.49075955 25.50357342]
 ...
 [33.33919751 33.61287414 33.65191763 33.67418015 33.67907881]
 [35.01498882 35.3159683  35.320376   35.32682329 35.33360739]
 [56.80942312 57.07062242 57.11370643 57.12230538 57.13718878]]
[[  0   7 310  22  99]
 [  1  83  19  16  60]
 [  2   7 310 301 124]
 ...
 [314 310 311   7 304]
 [315 305 304 306 303]
 [316 258 300   7 310]]


In [392]:
a=nbrs_fit.kneighbors_graph(counts).toarray()
a

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [393]:
elton

,URI,name,text,tfidf,count
303,<http://dbpedia.org/resource/Elton_John>,Elton John,sir elton hercules john cbe born reginald kenn...,"{'05': 0.0, '0fer': 0.0, '10': 0.0304392031560...","{'05': 0, '0fer': 0, '10': 1, '100': 3, '1000'..."


In [395]:
nbrs_fit.kneighbors(counts[[303]])

(array([[0.        , 1.05510969, 1.10523737, 1.11980056, 1.13304216]]),
 array([[303, 306, 304, 305, 315]], dtype=int64))

In [396]:
data.loc[[306]]   # Elton john most related to rod stewart

,URI,name,text,tfidf,count
306,<http://dbpedia.org/resource/Rod_Stewart>,Rod Stewart,roderick david rod stewart cbe born 10 january...,"{'05': 0.0, '0fer': 0.0, '10': 0.0613692988072...","{'05': 0, '0fer': 0, '10': 2, '100': 5, '1000'..."


In [397]:
victoria

,URI,name,text,tfidf,count
314,<http://dbpedia.org/resource/Victoria_Beckham>,Victoria Beckham,victoria caroline beckham ne adams born 17 apr...,"{'05': 0.0, '0fer': 0.0, '10': 0.0345294788899...","{'05': 0, '0fer': 0, '10': 1, '100': 0, '1000'..."


In [398]:
nbrs_fit.kneighbors(counts[[314]])

(array([[0.        , 1.12637845, 1.16609387, 1.18513652, 1.21663531]]),
 array([[314, 302, 311, 312, 154]], dtype=int64))

In [400]:
data.loc[[302]]  # victoria most related to david beckham 

,URI,name,text,tfidf,count
302,<http://dbpedia.org/resource/David_Beckham>,David Beckham,david robert joseph beckham obe bkm born 2 may...,"{'05': 0.0, '0fer': 0.0, '10': 0.0, '100': 0.0...","{'05': 0, '0fer': 0, '10': 0, '100': 2, '1000'..."
